# 导入文件

In [36]:
import pandas as pd
import pathlib
base_path = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls")
excel_file = pd.ExcelFile(base_path)
excel_file.sheet_names

['Sheet1', 'Sheet2', 'Sheet3']

In [58]:
import json
with open('config.json', 'r', encoding='utf-8') as f:
    CONF_DATA = json.load(f)
    BANK_CONF = CONF_DATA['bank_conf']

# 自定义函数

In [59]:
%run corelibs.py

# 执行

In [60]:
_local_bank_conf = BANK_CONF['天津银行账户'] #缓存本地配置
df = excel_file.parse(sheet_name=0, header=0, skiprows=0, dtype=str, keep_default_na='')
if 'merged_2cols' in _local_bank_conf:
    for _key, _value_list in _local_bank_conf['merged_2cols'].items():
        merge_2cols(df, _key, _value_list[0], _value_list[1])
df = df.reindex(columns=_local_bank_conf['cols_new_order'])
df

,开户名称,证件号码,电话,地址,客户账号,卡号,余额,开户日期,开户机构名称,账户状态,销户日期,销户机构名称,销户机构号,所属银行,是否开通网易,U盾号,邮编,短信捆绑电话,网银电话
0,崔家旭,130629198408220017,t1,,568001013102950620,623574256821695403,0.00,2016-06-30 00:00:00,天津银行滨海分行营业部,H-待激活,,,,天津银行滨海分行营业部,,,,t1,
1,张凯,120102197301301710,t2,,572001013103502425,623574257220661392,0.00,2014-12-09 00:00:00,天津银行第五中心支行营业部,A-活动,,,,天津银行第五中心支行营业部,,,,,t2
2,,,1,,,,,,,,,,,,,,,1,1
3,,,111222,,,,,,,,,,,,,,,111,222


# 杂项

In [38]:
yy=df[['短信捆绑电话', '网银电话']].copy()
a = yy['短信捆绑电话'] == yy['网银电话']
yy

,短信捆绑电话,网银电话
0,t1,
1,,t2
2,1,1
3,111,222


In [39]:
yy['短信捆绑电话'] = yy['短信捆绑电话']+yy['网银电话']
yy['短信捆绑电话'][a] = yy['网银电话']
yy

,短信捆绑电话,网银电话
0,t1,
1,t2,t2
2,1,1
3,111222,222


In [33]:
sum(yy['短信捆绑电话'], yy['网银电话'])

0      t11
1    t2t11
2     1t11
Name: 网银电话, dtype: object

In [ ]:
import dtale as dt
dt.show(dd, open_browser=True)

In [14]:
df.dtypes

所属银行      object
证件号码      object
开户名称      object
客户账号      object
卡号        object
余额        object
开户日期      object
开户机构名称    object
是否开通网易    object
U盾号       object
短信捆绑电话    object
网银电话      object
地址        object
邮编        object
销户日期      object
销户机构号     object
销户机构名称    object
账户状态      object
dtype: object